# **Mini curso SBPC 20255 - Caderno dia 4**

**ONCOGENÔMICA: FUNDAMENTOS, AVANÇOS E DESAFIOS NA PESQUISA DO CÂNCER**

Neste treinamento abordaremos aplicações práticas de bioinformática para análise de variantes genéticas, destacando sua importância na pesquisa, no diagnóstico e no tratamento do câncer através dos seguintes tópicos:
  
Dia 1:  

  - Aula teórica
  
Dia 2:
1. Apresentação do Google Colab
2. Instalando bibliotecas e o gerenciador mamba através do PIP
3. Download do genoma de referência e sequências de interesse
4. Análise de qualidade de dados em sequenciamento Illumina - formato Fastq  

Dia 3:
5. Alinhamento das leituras de sequenciamento contra um genoma de referência
6. Visualização de alinhamento  

#**Dia 4:**
7. Chamada de Variantes
8. Anotação de variantes

Use Ctrl+Enter para executar o conteúdo de uma 'célula' de código

Use Shift+Enter para executar código selecionado pelo mouse

Para começar, vamos instalar algumas bibliotecas e pacotes que serão usados ao longo do minicurso

💬 ❗ Para continuar o treinamento, faremos algumas instalações no google colab novamente e copiaremos alguns arquivos gerados no dia anterior.

In [ ]:
# Usando instalador de bibliotecas do Python (pip) para instalar o conda/mamba.
# Conda/Mamba é uma ferramenta que permite o gerenciamento de pacotes e ambientes virtuais.
# Que é um espaço isolado onde você pode instalar e executar ferramentas específicas para um projeto
!pip install --upgrade --force-reinstall zstandard
!pip install -q condacolab

!sed -i '/cudatoolkit/d' /usr/local/conda-meta/pinned

#Código em Python
import condacolab
condacolab.install()

#Instala algumas dependências necessárias para os próximos passos
#conda-forge é um canal comunitário em que ficam ferramentas e pacotes
#bioconda é um canal especializado em bioinfomática
!mamba install -c conda-forge curl --quiet

# Criando novo ambiente conda para instalar o git LFS.
!mamba create -n gitlfs
!mamba activate gitlfs
!mamba install -c bioconda git-lfs --quiet

# Instalando o plugin dp LFS no git local
!git lfs install

# Baixando sequência de interesse
!git --version
!git clone https://github.com/oncogensus/Minicurso_SBPC2025.git ./brca1

# Descompactando arquivos do dia 4
!mkdir refGen aligned
!cat brca1/Dia4/Dia4.tar.gz.part* > brca1/Dia4/Dia4.tar.gz
!tar -xzvf brca1/Dia4/Dia4.tar.gz -C brca1/Dia4/
!mv brca1/Dia4/Dia4_ORI/* refGen/
!cp ./brca1/Dia4/chr17_GRCh38.fasta.fai ./refGen/
!cp ./brca1/Dia4/clinvar_mini.vcf.* ./brca1/
!cp ./brca1/Dia4/aligned/* ./aligned/


# No passo anterior são baixados apenas apontadores dos arquivos grandes. Abaixo baixa os arquivos de fato.
!git pull lfs

# Baixando o ANNOVAR e amostras do 1000 Genomes
!wget -P brca1 https://github.com/oncogensus/treinamento_UFBA/raw/main/annovar.latest.tar.gz
!wget -P brca1 https://github.com/oncogensus/treinamento_UFBA/raw/main/samples2annot.vcf.gz

# Instalando o GATK4
!mamba create -n gatk
!mamba activate gatk
!mamba install -c bioconda gatk4 --quiet

#**7. Chamada de Variantes**

💬 ❗ Para a chamada de variantes usaremos o GATK, com a função HaplotypeCaller, usando o genoma de referência e as sequências de interesse previamente alinhadas contra o genômade referência.

Para mais informações sobre o GATK, clique nos links abaixo:

* [GATK/Broad Institute](https://gatk.broadinstitute.org/hc/en-us)
* [GATK - Documentação](https://gatk.broadinstitute.org/hc/en-us/categories/360002310591)




💬 ❗ Criaremos um arquivo de dicionário que contém informações sobre as referências genômicas usadas em uma análise. Ele é necessário para várias ferramentas do GATK, que requerem a ordenação e a estruturação correta da sequência de referência com a qual estão sendo comparadas as variantes genômicas.

In [ ]:
# Criando o dicionário para a referência
!gatk CreateSequenceDictionary \
    -R /content/refGen/chr17_GRCh38.fasta \
    -O /content/refGen/chr17_GRCh38.dict


💬 ❗ Finalmente faremos as chamadas de variantes para as 3 amostras que temos contra a referência. A execução leva em torno de 5 minutos.

In [ ]:
# Criando a pasta variants
!mamba activate gatk
!echo -e "\nCriando a pasta varients\n"
!mkdir variants
!echo -e "\nChamando variantes para BRCA1_WT\n"
!pwd
#!touch ./variants/BRCA1_WT.g.vcf.gz
!gatk HaplotypeCaller -R /content/refGen/chr17_GRCh38.fasta -I ./aligned/BRCA1_WT_mapped.bam -O ./variants/BRCA1_WT.g.vcf.gz -ERC GVCF
!gatk HaplotypeCaller -R /content/refGen/chr17_GRCh38.fasta -I ./aligned/BRCA1_185delAG_mapped.bam -O ./variants/BRCA1_185delAG.g.vcf.gz -ERC GVCF
!gatk HaplotypeCaller -R /content/refGen/chr17_GRCh38.fasta -I ./aligned/BRCA1_c.5266dupC_mapped.bam -O ./variants/BRCA1_c.5266dupC.g.vcf.gz -ERC GVCF


💬 ❗ Abaixo combinaremos as chamadas de variantes das amostras em um cohort.

In [ ]:
# Combinar os GVCF em um arquivo
!gatk CombineGVCFs -R /content/refGen/chr17_GRCh38.fasta -V ./variants/BRCA1_WT.g.vcf.gz -V ./variants/BRCA1_185delAG.g.vcf.gz -V ./variants/BRCA1_c.5266dupC.g.vcf.gz -O ./variants/cohort.g.vcf.gz


💬 ❗ Vamos transformar arquivos de variantes no formato GVCF (Genomic Variant Call Format) em um VCF final que contém as variantes genotipadas.

In [ ]:
#  Genotipagem de variantes
!gatk GenotypeGVCFs -R /content/refGen/chr17_GRCh38.fasta -V ./variants/cohort.g.vcf.gz -O ./variants/cohort.vcf.gz


💬 ❗ Agora vamos aplicar alguns filtros de qualidade das chamadas de variantes.

Abaixo algumas legendas:

*   QD (QualByDepth): é calculado como a razão entre a qualidade de chamada de variante (QUAL) e a profundidade de cobertura no local da variante (DP).
*   FS (Fisher Strand Bias): é uma métrica que mede a presença de viés de strand em torno de uma chamada de variante. Ocorre quando as variantes são detectadas preferencialmente em um dos strands (fitas) do DNA durante o sequenciamento
*   MQ (Mapping Quality): representa a qualidade de mapeamento das leituras (reads) que suportam a variante.





In [ ]:
# Aplicando alguns Filtros de qualidade
!gatk VariantFiltration -R /content/refGen/chr17_GRCh38.fasta -V ./variants/cohort.vcf.gz -O ./variants/cohort_filtered.vcf.gz --filter-expression "QD < 2.0 || FS > 60.0 || MQ < 40.0" --filter-name "my_snp_filter"


💬 ❗ Verificando o arquivo com as variantes chamadas.

Algumas informações importantes:

* GT (Genotype): Representa o genótipo da amostra para aquela posição. Por exemplo, "0/0" indica homozigoto para o alelo de referência. Já "0/1" indica que a amostra é heterozigota (uma cópia de cada alelo), enquanto "1/1" indica homozigoto para o alelo alternativo.

* AD (Allelic Depth): Mostra as contagens de leitura para os alelos de referência e alternativo. Por exemplo, AD=10,5 significa que foram observadas 10 leituras para o alelo de referência e 5 leituras para o alelo alternativo.

* DP (Depth of Coverage): Representa o número total de leituras que cobrem a posição variante, ou seja, a soma dos valores de AD. No exemplo anterior, seria 15 (10 leituras de referência + 5 leituras alternativos).

* GQ (Genotype Quality): Indica a confiança da chamada do genótipo. Geralmente, quanto maior o valor, maior a confiabilidade do genótipo chamado.

* PL (Phred-scaled Likelihoods): Indica as probabilidades relativas de cada genótipo possível (homozigoto referência, heterozigoto, homozigoto alternativo) em uma escala Phred . Quanto menor o valor, mais provável é o genótipo. Por exemplo, PL=0,50,100 indica que o genótipo homozigoto referência é o mais provável (PL=0).

In [ ]:
!zcat ./variants/cohort_filtered.vcf.gz

#**8. Anotação de variantes**

💬 ❗ Existem várias bases de dados com informações de variantes conhecidas. Temos também ferramentas como o ANNOVAR que é capaz de baixar essas bases e fazer anotações nos nossos arquivos de variantes, adicionando informações importantes de forma conveniente. Usaremos o bcftools para adicionar anotações baseadas na base de dados do Clinvar.

Para mais informações:

* [bcftools](https://github.com/samtools/bcftools)
* [ANNOVAR](https://annovar.openbioinformatics.org/en/latest/)


In [ ]:
# Criando ambiente mamba para o BCFTOOLS e instalando a aplicação
!mamba create -n bcftools
!mamba activate bcftools
!mamba install -c bioconda bcftools --quiet

💬 ❗ Vamos verificar o formato da minibase do clinvar, que foi baixadas do github na etapa 3.

In [ ]:
# Verificando minibase do Clinvar

!zcat ./brca1/clinvar_mini.vcf.gz

💬 ❗ Agora faremos a anotação do nosso VCF gerado com o bcftools e nossa minibase do Clinvar que foi baixadas do github na etapa 3.

In [ ]:
!echo -e "\nCriando a pasta annotated"
!mkdir annotated
!echo -e "\nAnotando o arquivo cohort_filtered.vcf.gz"
# Anotação com o bcftools e a minibase
!bcftools annotate -a ./brca1/clinvar_mini.vcf.gz -c ID,INFO ./variants/cohort_filtered.vcf.gz -o ./annotated/cohort_annotated.vcf


💬 ❗ Verificando o vcf com as anotações.

In [ ]:
# Verificando vcf com as anotações da minibase do Clinvar.
!cat ./annotated/cohort_annotated.vcf

💬 ❗ No começo do treinamento baixamos vários arquivos do github, incluindo ANNOVAR e um VCF com três amostras do 1000 Genomes (ambos estão na pasta brca1). Vamos descompactar o ANNOVAR.

In [ ]:
# Descompactando o ANNOVAR.
!tar -xvzf ./brca1/annovar.latest.tar.gz
!ls -lh ./annovar/
!echo -e "\n### Verificando bases que já vem no ANNOVAR dentro da pasta humandb ###\n"
!ls -lhSr ./annovar/humandb/

💬 ❗ O ANNOVAR facilita o download de novas bases de dados para anotação de vcf. Para verificar as bases disponíveis para os genomas de referência, acesse [a página de downloads do ANNOVAR.](https://annovar.openbioinformatics.org/en/latest/user-guide/download/)



In [ ]:
# Usando o ANNOVAR para baixar as bases Clinvar e AbraOM
!./annovar/annotate_variation.pl -buildver hg38 -downdb -webfrom annovar clinvar_20240917 ./annovar/humandb/
!echo -e "\n ### Verificando as bases baixadas ###"
!ls -lh ./annovar/humandb/

💬 ❗ Agora usaremos o ANNOVAR para anotar o VCF com as três amostras do 1000 Genomes usandos as bases baixadas.

In [ ]:
!gunzip ./brca1/samples2annot.vcf.gz
!./annovar/table_annovar.pl ./brca1/samples2annot.vcf ./annovar/humandb/ -buildver hg38 -out ./annotated/samples_clinvar -remove -protocol clinvar_20240917 -operation f -nastring . -vcfinput

**Explicando o comando:**  

!./annovar/table_annovar.pl \      # Executa o script table_annovar.pl do ANNOVAR  

./brca1/samples2annot.vcf \        # Arquivo VCF de entrada para anotação  

./annovar/humandb/ \               # Diretório de bancos de dados do ANNOVAR  

-buildver hg38 \                   # Build do genoma: hg38  

-out ./annotated/samples_clinvar \ # Prefixo do arquivo de saída  

-remove \                          # Remove arquivos intermediários  

-protocol clinvar_20240917 \       # Banco de dados: ClinVar (versão 17/09/2024)  

-operation f \                     # Tipo de anotação: baseada em gene  

-nastring . \                      # Representa dados faltantes como "."  

-vcfinput                          # Entrada no formato VCF


💬 ❗ Finalmente vamos verificar variantes patogênicas anotadas pelo ANNOVAR com a base do Clinvar.

In [ ]:
!grep "Pathogenic" ./annotated/samples_clinvar.hg38_multianno.vcf

#OBRIGADO!!